In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [2]:
import json

# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = '../dataset/prep/test_json.json'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        # 读取并解析 JSON 数据
        test_data = json.load(file)
        # 打印读取的数据
        print(test_data[0])
except FileNotFoundError:
    print(f"文件 {file_path} 未找到。")
except json.JSONDecodeError:
    print(f"文件 {file_path} 不是有效的 JSON 格式。")

{'ID': '1405', 'instruction': "\n请在从病历中患者的详细情况描述（包括现病史、主诉、四诊信息等），判断出患者所患的证型和疾病，并推荐当前患者的中药处方，你需要输出一个json，格式为{'疾病':'', '证型':'', '处方': ''}\n请注意你生成的json的疾病、症型、处方只能是以下范围的\n疾病范围：['胸痹心痛病', '心衰病', '眩晕病', '心悸病']\n证型范围：['气虚血瘀证', '痰瘀互结证', '气阴两虚证', '气滞血瘀证', '肝阳上亢证', '阴虚阳亢证', '痰热蕴结证', '痰湿痹阻证', '阳虚水停证', '肝肾阴虚证']\n处方范围：['冬瓜皮', '沉香', '茜草炭', '浮小麦', '炙甘草', '炒白扁豆', '砂仁', '合欢花', '北刘寄奴', '炒六神曲', '炒决明子', '益母草', '酒苁蓉', '炒僵蚕', '稀莶草', '秦艽', '黄酒', '瞿麦', ' 白鲜皮', '熟地黄', '扁蓄', '诃子肉', '煅牡蛎', '鸡血藤', '党参', '瓜蒌', '莲子', '酒五味子', '金钱草', '法半夏', '北败酱草', '花椒', '吴茱萸(粉)', '桑白皮', '茯神', '桂枝', '降香', '制远志', '琥珀', '佛手', '麦芽', '水红花子', '金银花', '马鞭草', '半枝莲', '炮姜', '生酸枣仁', '盐补骨脂', '炒瓜蒌子', '珍珠母', '乌药', '茵陈', '地肤子', '酸枣仁', '槟榔', '大青叶', '人参片', '麸煨肉豆蔻', '蛤蚧', '路路通', '蝉蜕', '马勃', '香橼', '络石藤', '狗脊', '蜈蚣', '制川乌', '白扁豆花', '麻黄', '射干', '厚朴', '蜂蜜', '柏子仁', '炒谷芽', '蜜百合', ' 石菖蒲', '白薇', '续断', '炒川楝子', '黄连片', '绵萆薢', '鹿角胶', '翻白草', '羚羊角粉', '天麻', '山慈菇', '菊花', '炒芥子', '墨旱莲', '蜜枇杷叶', '川芎', '酒大黄', '焦山楂', '红曲', '山药', '牡蛎', '海藻', '夏枯草', '白前', '白芍', '茯苓皮', '煅自然铜'

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/root/autodl-tmp/self-llm/model/Qwen/Qwen2.5-7B-Instruct'
lora_path = './output/Qwen2.5_instruct_lora/checkpoint-100' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm

# 假设 test_data、tokenizer 和 model 已经定义
for d in tqdm(test_data[150:], desc="Processing test data"):
    prompt = d['input']
    inputs = tokenizer.apply_chat_template([{"role": "user", "content": d['instruction']},{"role": "user", "content": prompt}],
                                           add_generation_prompt=True,
                                           tokenize=True,
                                           return_tensors="pt",
                                           return_dict=True
                                           ).to('cuda')
    
    
    gen_kwargs = {"max_length": 8000, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        d['output'] = tokenizer.decode(outputs[0], skip_special_tokens=True)

Processing test data:  14%|█▍        | 7/50 [00:51<05:07,  7.14s/it]

In [ ]:
# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = '../dataset/res/test_json4.json'
try:
    with open(file_path, 'w', encoding='utf-8') as file:
        # 将 list 转换为 JSON 格式并写入文件
        json.dump(test_data, file, ensure_ascii=False, indent=4)
    print(f"数据已成功保存到 {file_path}")
except Exception as e:
    print(f"保存文件时出现错误: {e}")